In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.preprocessing.filters import Filters as filters
from ptrail.features.kinematic_features import KinematicFeatures as spatial
from ptrail.preprocessing.interpolation import Interpolation
import ptrail.utilities.constants as const

# Reading the gulls dataset and converting to PTRAILDataFrame.
# Also, lets, print the first 5 points of the dataset to
# see how the dataframe looks.
gulls = pd.read_csv('./data/starkey.csv')
gulls = PTRAILDataFrame(gulls,
                        latitude='lat',
                        longitude='lon',
                        datetime='DateTime',
                        traj_id='Id',
                        rest_of_columns=[])
gulls.head()

lat         lon  StarkeyTime    GMDate  \
traj_id   DateTime                                                            
880109D01 1995-04-13 13:40:06  45.239682 -118.533204    229902006  21:40:06   
          1995-04-15 12:16:15  45.250521 -118.530438    230069775  20:16:15   
          1995-04-15 21:39:38  45.247943 -118.541455    230103578  05:39:38   
          1995-04-16 03:32:14  45.247429 -118.539530    230124734  11:32:14   
          1995-04-16 04:08:28  45.247117 -118.542579    230126908  12:08:28   

                                 GMTime   LocDate   LocTime  RadNum Species  \
traj_id   DateTime                                                            
880109D01 1995-04-13 13:40:06  19950413  19950413  13:40:06     409       D   
          1995-04-15 12:16:15  19950415  19950415  12:16:15     409       D   
          1995-04-15 21:39:38  19950416  19950415  21:39:38     409       D   
          1995-04-16 03:32:14  19950416  19950416  03:32:14     409       D   
          1995-04-16 04:08:28  19950416  19950416  04:08:28     409       D   

                                 UTME     UTMN  Year  Grensunr  Grensuns  \
traj_id   DateTime                                                         
880109D01 1995-04-13 13:40:06  379662  5010734    95  13:13:00  02:39:00   
          1995-04-15 12:16:15  379895  5011927    95  13:09:00  02:41:00   
          1995-04-15 21:39:38  379039  5011656    95  13:07:00  02:43:00   
          1995-04-16 03:32:14  379188  5011581    95  13:07:00  02:43:00   
          1995-04-16 04:08:28  378938  5011567    95  13:07:00  02:43:00   

                               Obswt  
traj_id   DateTime                    
880109D01 1995-04-13 13:40:06   1.47  
          1995-04-15 12:16:15   1.59  
          1995-04-15 21:39:38   1.34  
          1995-04-16 03:32:14   1.50  
          1995-04-16 04:08:28   1.34

In [12]:
a = gulls.reset_index().describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9], include='all', datetime_is_numeric=True)
a

,traj_id,DateTime,lat,lon,StarkeyTime,GMDate,GMTime,LocDate,LocTime,RadNum,Species,UTME,UTMN,Year,Grensunr,Grensuns,Obswt
count,287136,287136,287136.000000,287136.000000,2.871360e+05,287136,2.871360e+05,2.871360e+05,287136,287136.000000,287136,287136.000000,2.871360e+05,287136.000000,287136,287136,287136.000000
unique,253,NaN,NaN,NaN,NaN,79481,NaN,NaN,79481,NaN,3,NaN,NaN,NaN,73,77,NaN
top,910313E37,NaN,NaN,NaN,NaN,02:24:41,NaN,NaN,20:15:51,NaN,E,NaN,NaN,NaN,12:06:00,03:49:00,NaN
freq,4007,NaN,NaN,NaN,NaN,21,NaN,NaN,21,NaN,166885,NaN,NaN,NaN,34340,24548,NaN
mean,NaN,1994-11-18 15:57:57.838536320,45.252101,-118.563552,2.172953e+08,NaN,1.994497e+07,1.994497e+07,NaN,229.597981,NaN,377310.246904,5.012146e+06,94.436800,NaN,NaN,1.554205
min,NaN,1993-05-06 17:00:21,45.188970,-118.610208,1.688256e+08,NaN,1.993051e+07,1.993051e+07,NaN,8.000000,NaN,373705.000000,5.005110e+06,93.000000,NaN,NaN,1.140000
10%,NaN,1993-05-31 06:44:31,45.211269,-118.593836,1.709523e+08,NaN,1.993053e+07,1.993053e+07,NaN,54.000000,NaN,374952.000000,5.007578e+06,93.000000,NaN,NaN,1.370000
25%,NaN,1993-07-11 13:50:12.500000,45.226470,-118.581813,1.745202e+08,NaN,1.993071e+07,1.993071e+07,NaN,120.000000,NaN,375913.000000,5.009288e+06,93.000000,NaN,NaN,1.440000
50%,NaN,1994-07-17 17:27:56.500000,45.250933,-118.566530,2.065833e+08,NaN,1.994072e+07,1.994072e+07,NaN,206.000000,NaN,377060.000000,5.012016e+06,94.000000,NaN,NaN,1.530000
75%,NaN,1996-05-04 03:20:08.250000,45.278305,-118.544923,2.633016e+08,NaN,1.996050e+07,1.996050e+07,NaN,346.000000,NaN,378754.000000,5.015057e+06,96.000000,NaN,NaN,1.650000


In [13]:
a.transpose()

,count,unique,top,freq,mean,min,10%,25%,50%,75%,90%,max,std
traj_id,287136,253,910313E37,4007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DateTime,287136,NaN,NaN,NaN,1994-11-18 15:57:57.838536320,1993-05-06 17:00:21,1993-05-31 06:44:31,1993-07-11 13:50:12.500000,1994-07-17 17:27:56.500000,1996-05-04 03:20:08.250000,1996-06-28 17:50:53,1996-08-15 15:52:06,NaN
lat,287136.0,NaN,NaN,NaN,45.252101,45.18897,45.211269,45.22647,45.250933,45.278305,45.291484,45.314546,0.030609
lon,287136.0,NaN,NaN,NaN,-118.563552,-118.610208,-118.593836,-118.581813,-118.56653,-118.544923,-118.530427,-118.504556,0.023315
StarkeyTime,287136.0,NaN,NaN,NaN,217295330.16489,168825628.0,170952271.0,174520212.5,206583257.0,263301608.25,268102923.0,272245926.0,36618455.314862
GMDate,287136,79481,02:24:41,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GMTime,287136.0,NaN,NaN,NaN,19944971.478035,19930507.0,19930531.0,19930711.0,19940718.0,19960504.0,19960629.0,19960815.0,11479.963229
LocDate,287136.0,NaN,NaN,NaN,19944970.352791,19930506.0,19930531.0,19930711.0,19940717.0,19960504.0,19960628.0,19960815.0,11480.030811
LocTime,287136,79481,20:15:51,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RadNum,287136.0,NaN,NaN,NaN,229.597981,8.0,54.0,120.0,206.0,346.0,400.0,450.0,130.205624
